# buy and hold 전략
1. 구매 후 일정 기간이 지나면 판매

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../csv/AAPL.csv', index_col='Date')
df

In [ ]:
# 결측치를 확인 
df.info()

In [ ]:
# Volume의 데이터가 결측치인 데이터를 출력
df.loc[~df["Volume"].isna()]

In [ ]:
df.drop(["1981-08-10", "1981-08-21"], axis=0)

In [ ]:
# 결측치와 양의 무한대, 음의 무한대의 값을 제외한 데이터만 출력
df.loc[df['Open'] != -np.inf]
df.loc[df['Open'] != np.inf]
df.loc[~df['Open'].isna()]

In [ ]:
df.loc[
    (df['Open'] != -np.inf) & (df['Open'] != np.inf) & (~df['Open'].isna())
]

In [ ]:
flag = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)

In [34]:
df = df.loc[flag]

In [ ]:
# df의 종가를 제외한 데이터를 모두 삭제
df[['Close']]

In [40]:
price_df = df.loc[:, ['Close']]

In [ ]:
# 일별 수익율 = (오늘의 종가 - 전날의 종가) / 전날의 종가
# 전 행의 데이터를 로드하려면? shift(n)
# case1
(price_df['Close'] - price_df['Close'].shift(1)) / price_df['Close'].shift(1)


In [ ]:
# case2
price_df['Close'].diff() / price_df['Close'].shift()


In [49]:
# case3
price_df['daily_rtn'] = price_df['Close'].pct_change()

In [52]:
price_df

,Close,daily_rtn
Date,,
1980-12-12,0.513393,NaN
1980-12-15,0.486607,-0.052174
1980-12-16,0.450893,-0.073394
1980-12-17,0.462054,0.024753
1980-12-18,0.475446,0.028984
...,...,...
2019-06-18,198.449997,0.023518
2019-06-19,197.869995,-0.002923
2019-06-20,199.460007,0.008036


In [53]:
# 일별 수익율을 가지고 총 수익율 계산
test_df = price_df.iloc[300:600]
test_df

,Close,daily_rtn
Date,,
1982-02-24,0.328125,0.006849
1982-02-25,0.325893,-0.006802
1982-02-26,0.325893,0.000000
1982-03-01,0.328125,0.006849
1982-03-02,0.328125,0.000000
...,...,...
1983-04-25,0.868304,-0.046568
1983-04-26,0.892857,0.028277
1983-04-27,0.883929,-0.009999


In [55]:
rtn = 1
# rtn * (1 + daily_rtn)

for i in test_df['daily_rtn'].tolist():
    rtn = rtn * (1 + i)

print(rtn)


2.767122951398157


In [56]:
(1 + test_df['daily_rtn']).cumprod()

Date
1982-02-24    1.006849
1982-02-25    1.000000
1982-02-26    1.000000
1982-03-01    1.006849
1982-03-02    1.006849
                ...   
1983-04-25    2.664384
1983-04-26    2.739724
1983-04-27    2.712329
1983-04-28    2.739724
1983-04-29    2.767123
Name: daily_rtn, Length: 300, dtype: float64

In [64]:
# (매도한 날의 종가 - 매수한 날의 종가) / 매수한 날의 종가
(test_df.iloc[-1]['Close'] - test_df.iloc[0]['Close']) / test_df.iloc[0]['Close']

1.7483001904761903

In [65]:
## buyandhold 함수로 생성
## 매개변수를 3개
## 데이터프레임 , 구매한 날, 판매한 날
def bnh(_df, _start, _end):
    # index의 데이터를 시계열로 변경
    _df.index = pd.to_datetime(_df.index, format='%Y-%m-%d')
    # 결측치와 무한대의 값을 제외시키는 부분
    _df = _df.loc[ ~_df.isin([np.nan, np.inf, -np.inf]).any(axis=1) ]
    # 종가를 제외한 나머지 컬럼을 모두 삭제
    _df = _df[['Close']]
    # 일별 수익율 컬럼를 생성
    _df['daily_rtn'] = _df['Close'].pct_change()
    # 구매한 날짜와 판매한 날짜를 기준으로 데이터를 필퍼( 행의 조건으로 필터 )
    _df = _df.loc[_start : _end]
    # 누적 수익율 컬럼을 생성
    _df['st_rtn'] = (1 + _df['daily_rtn']).cumprod()

    return _df

In [66]:
# 문자열 데이터를 시계열 변경
from datetime import datetime

In [ ]:
df = pd.read_csv("../csv/AMZN.csv", index_col='Date')
df

In [70]:
start = '20000102'
end = '20151231'
# 문자열을 시계열 변경
start = datetime.strptime(start, '%Y%m%d').isoformat()
end = datetime.strptime(end, '%Y%m%d').isoformat()

In [71]:
bnh(df, start, end)

,Close,daily_rtn,st_rtn
Date,,,
2000-01-03,89.375000,0.174056,1.174056
2000-01-04,81.937500,-0.083217,1.076355
2000-01-05,69.750000,-0.148741,0.916256
2000-01-06,65.562500,-0.060036,0.861248
2000-01-07,69.562500,0.061010,0.913793
...,...,...,...
2015-12-24,662.789978,-0.001371,8.706601
2015-12-28,675.200012,0.018724,8.869622
2015-12-29,693.969971,0.027799,9.116190
